Data stored at 

* /ocean/rich/home/SoG/ios/che/
* /ocean/rich/home/SoG/ios/che2


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import glob
import os
import datetime
import pandas as pd
import netCDF4 as nc

from salishsea_tools.nowcast import figures
from salishsea_tools import viz_tools
%matplotlib inline

In [2]:
#Load NEMO grid
f = '/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc'
grid = nc.Dataset(f)

In [3]:
def read_IOS(filename):
    """Funtion to read IOS data in filename"""
    filehandle = open(filename,'rt')
    ss_depth = [] #depth
    ss_sal = [] #salinity
    ss_temp = [] #temperature
    with filehandle as data:
            headerover = False
            for line in data:
                if headerover:
                    break
                    word = line.split()
                    ss_depth.append(float(word[0]))
                    ss_sal.append( float(word[5]))
                    ss_temp.append(float(word[1]))
                elif line[0:14] == "*END OF HEADER":
                    headerover = True
                    next_line = filehandle.next()
                    wordd = next_line.split()
                    for next_line in data:
                        ss_depth.append(float(wordd[1]))
                        ss_temp.append(float(wordd[2]))
                        ss_sal.append(float(wordd[5]))
                        wordd = next_line.split()
                    ss_depth.append(float(wordd[1]))
                    ss_temp.append(float(wordd[2]))
                    ss_sal.append(float(wordd[5]))
                else:
                    word = line.split()
                    if len(word) > 1:
                        if word[0] == "LATITUDE" and word[1] !='2':
                            lat = float(word[2])+ float(word[3])/60
                        if word[0] == "LONGITUDE" and word[1] !='2':
                            lon = -(float(word[2])+ float(word[3])/60)
                        if word[0] == "START":
                            string = word[4] +' ' + word[5]
                            time = datetime.datetime.strptime(string,'%Y/%m/%d %H:%M:%S.%f')
    return lat,lon,time,ss_depth,ss_sal,ss_temp

loop through the first directory

In [4]:
ios_dir = '/ocean/rich/home/SoG/ios/che/'
files = glob.glob(os.path.join(ios_dir,'*'))
files.sort(key=os.path.basename)

In [5]:
times = []
lats=[]
lons=[]
years = []
months = []
depths = []
sals = []
temps = []
counter = 0
for filename in files:
    lat,lon,time, depth,sal, temp = read_IOS(filename)
    times.append(time)
    lats.append(lat)
    lons.append(lon)
    years.append(time.year)
    months.append(time.month)
    depths.append(depth)
    sals.append(sal)
    temps.append(temp)
    counter=counter+1
#Check number of files looked at
print counter
print len(temps)

459
459


loop through the second directoy

In [6]:
ios_dir = '/ocean/rich/home/SoG/ios/che2/'
files = glob.glob(os.path.join(ios_dir,'*'))
files.sort(key=os.path.basename)

In [7]:
counter=0
for filename in files:
    lat,lon,time, depth,sal, temp = read_IOS(filename)
    times.append(time)
    lats.append(lat)
    lons.append(lon)
    years.append(time.year)
    months.append(time.month)
    depths.append(depth)
    sals.append(sal)
    temps.append(temp)
    counter=counter + 1
print counter
print len(files)

477
477


In [9]:
data = pd.DataFrame({'year': years, 'date': times, 'lat':lats, 'lon': lons,'month':months,\
                    'depth':depths, 'salinity': sals, 'temperature': temps})

In [10]:
data

,date,depth,lat,lon,month,salinity,temperature,year
0,1980-04-18 11:45:00,"[6.9, 17.8, 19.4, 30.4, 49.9, 74.3, 100.4, 126.3]",48.903000,-123.230000,4,"[17.7, 2.04, 0.98, -99.0, -99.0, -99.0, -99.0,...","[11.2, 26.4, 19.7, 24.7, 28.1, 26.9, 25.0, 24.9]",1980
1,1980-06-19 22:49:00,"[5.3, 12.4, 25.2, 29.3, 49.1, 99.2, 181.3]",49.367000,-123.750000,6,"[4.0, 7.66, -99.0, -99.0, -99.0, -99.0, -99.0]","[4.3, 21.4, 26.6, 21.2, 31.4, 28.8, -99.0]",1980
2,1981-07-03 04:10:00,"[5.4, 15.3, 15.8, 33.5, 250.5]",49.367000,-123.917000,7,"[0.74, 3.42, 0.99, -99.0, -99.0]","[0.1, 11.4, 2.0, 27.5, -99.0]",1981
3,1982-05-20 12:12:00,"[8.8, 18.9, 30.3, 49.1, 75.1, 100.5, 101.8, 12...",49.317000,-123.750000,5,"[0.28, 0.15, -99.0, -99.0, -99.0, -99.0, -99.0...","[13.7, 24.8, 27.5, 27.5, 28.0, 27.1, 30.9, 23....",1982
4,1982-07-15 09:51:00,"[9.3, 20.2, 29.9, 51.7, 69.5]",49.352000,-123.615000,7,"[9.83, 0.19, -99.0, -99.0, -99.0]","[16.1, 20.8, 22.8, 22.2, 24.3]",1982
5,1982-07-16 01:21:00,"[12.0, 21.5, 30.5, 52.3, 75.4, 98.3, 99.6, 124...",49.250000,-123.943000,7,"[5.28, 0.61, -99.0, -99.0, -99.0, -99.0, -99.0...","[6.6, 21.2, 23.5, 28.0, 26.5, 28.1, 26.1, 29.1...",1982
6,1983-05-20 12:55:00,"[9.6, 97.9, 126.9, 138.5]",48.893000,-123.303000,5,"[0.81, 0.14, -99.0, -99.0]","[11.7, 14.6, 23.8, 23.5]",1983
7,1996-02-26 15:38:00,"[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0,...",48.862333,-123.080000,2,"[25.0, 23.6, 26.6, 23.2, 26.7, 26.6, 23.0, 22....","[-99.0, 29.0189, 29.3459, 29.4757, 29.5546, 29...",1996
8,1996-02-26 16:42:00,"[8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0]",48.918333,-123.179833,2,"[25.2, 23.9, 25.5, 25.2, 24.0, 23.5, 26.4, 23.8]","[25.0212, 28.8801, 29.0425, 29.1258, 29.5534, ...",1996
9,1996-02-26 17:47:00,"[10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0,...",48.976167,-123.272333,2,"[22.1, 25.4, 23.1, 25.6, 26.7, 22.6, 21.9, 27....","[25.5136, 28.055, 28.7888, 29.338, 29.4314, 29...",1996
